In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!git clone https://github.com/kakao/khaiii.git
!pip install cmake
!mkdir build
!cd build && cmake /content/khaiii
!cd /content/build/ && make all
!cd /content/build/ && make resource
!cd /content/build && make install
!cd /content/build && make package_python
!pip install /content/build/package_python

In [4]:
from khaiii import KhaiiiApi
import pandas as pd
import numpy as np
import json, re
from collections import Counter
from typing import *
from tqdm import tqdm

In [5]:
#train = pd.read_json("train.json", encoding = 'utf-8')
#val = pd.read_json("val.json", encoding = 'utf-8')
train = pd.read_json('/content/drive/My Drive/kakao_arena/train.json', encoding='utf-8')
val = pd.read_json('/content/drive/My Drive/kakao_arena/val.json', encoding='utf-8')

In [6]:
def re_sub(series: pd.Series) -> pd.Series:
    series = series.str.replace(pat=r'[ㄱ-ㅎ]', repl=r'', regex=True)  # ㅋ 제거용
    series = series.str.replace(pat=r'[^\w\s]', repl=r'', regex=True)  # 특수문자 제거
    series = series.str.replace(pat=r'[ ]{2,}', repl=r' ', regex=True)  # 공백 제거
    series = series.str.replace(pat=r'[\u3000]+', repl=r'', regex=True)  # u3000 제거
    return series

def flatten(list_of_list : List) -> List:
    flatten = [j for i in list_of_list for j in i]
    return flatten

def get_token(title: str, tokenizer)-> List[Tuple]:
    
    if len(title)== 0 or title== ' ':  # 제목이 공백인 경우 tokenizer에러 발생
        return []
    
    result = tokenizer.analyze(title)
    result = [(morph.lex, morph.tag) for split in result for morph in split.morphs]  # (형태소, 품사) 튜플의 리스트
    return result

def get_all_tags(df, column):
    tag_list = df[column].values.tolist()
    tag_list = flatten(tag_list)
    return tag_list

def lower(data):
    data_list=data.values.tolist()
    data_lower=[[j.lower() for j in i] for i in data_list ]
    return data_lower

In [7]:
train["lower_tags"]=lower(train["tags"])
train_lower_tags_set = set(flatten(train["lower_tags"]))

In [8]:
val["plylst_title"] = re_sub(val['plylst_title'])

In [9]:
val_song0 = val[val['songs'].map(len) == 0]

In [10]:
val_song0.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
8,[스트레스],80810,리듬타면서 빡시게 운동하자스트레스 날리자,[],127,2017-02-09 17:33:45.000
9,[],142007,기분 좋은 재즈와 함께 만드는 달달한 하루,[],0,2015-06-22 09:11:02.000
17,[생각나],2380,다시 생각나는 그 사람,[],6,2016-03-19 23:53:20.000
20,"[피아노, 이루마, 메로디]",149069,불면증엔 아름다운 뉴에이지 곡들,[],11,2019-12-15 19:15:21.000


In [11]:
candidate_tag = [i.split() for i in val_song0['plylst_title'].values.tolist()]
val_song0["candidate_tag"] = candidate_tag
#candidate_tag = lower(val_song0["candidate_tag"])  소문자처리

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
original_tag = []
new_tag = []
for i in tqdm(candidate_tag):
    o_tag = []
    n_tag = []
    for j in i :
        if j in train_lower_tags_set:
            o_tag.append(j)
        else :
            n_tag.append(j)
    original_tag.append(o_tag)
    new_tag.append(n_tag)

100%|██████████| 4379/4379 [00:00<00:00, 281648.43it/s]


In [13]:
val_song0["new_tag"] = new_tag
val_song0["original_tag"] = original_tag

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
val_song0.head()

,tags,id,plylst_title,songs,like_cnt,updt_date,candidate_tag,new_tag,original_tag
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000,[앨리스테이블],[앨리스테이블],[]
8,[스트레스],80810,리듬타면서 빡시게 운동하자스트레스 날리자,[],127,2017-02-09 17:33:45.000,"[리듬타면서, 빡시게, 운동하자스트레스, 날리자]","[리듬타면서, 빡시게, 운동하자스트레스]",[날리자]
9,[],142007,기분 좋은 재즈와 함께 만드는 달달한 하루,[],0,2015-06-22 09:11:02.000,"[기분, 좋은, 재즈와, 함께, 만드는, 달달한, 하루]","[재즈와, 만드는]","[기분, 좋은, 함께, 달달한, 하루]"
17,[생각나],2380,다시 생각나는 그 사람,[],6,2016-03-19 23:53:20.000,"[다시, 생각나는, 그, 사람]",[],"[다시, 생각나는, 그, 사람]"
20,"[피아노, 이루마, 메로디]",149069,불면증엔 아름다운 뉴에이지 곡들,[],11,2019-12-15 19:15:21.000,"[불면증엔, 아름다운, 뉴에이지, 곡들]",[불면증엔],"[아름다운, 뉴에이지, 곡들]"


In [15]:
del val_song0["candidate_tag"]

In [16]:
val_song0.head()

,tags,id,plylst_title,songs,like_cnt,updt_date,new_tag,original_tag
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000,[앨리스테이블],[]
8,[스트레스],80810,리듬타면서 빡시게 운동하자스트레스 날리자,[],127,2017-02-09 17:33:45.000,"[리듬타면서, 빡시게, 운동하자스트레스]",[날리자]
9,[],142007,기분 좋은 재즈와 함께 만드는 달달한 하루,[],0,2015-06-22 09:11:02.000,"[재즈와, 만드는]","[기분, 좋은, 함께, 달달한, 하루]"
17,[생각나],2380,다시 생각나는 그 사람,[],6,2016-03-19 23:53:20.000,[],"[다시, 생각나는, 그, 사람]"
20,"[피아노, 이루마, 메로디]",149069,불면증엔 아름다운 뉴에이지 곡들,[],11,2019-12-15 19:15:21.000,[불면증엔],"[아름다운, 뉴에이지, 곡들]"


### khaiii 이용 (colab 필요)

In [17]:
#train = pd.read_json('/content/drive/My Drive/kakao_arena/train.json', encoding='utf-8')
#val = pd.read_json('/content/drive/My Drive/kakao_arena/val.json', encoding='utf-8')
#val_song0 = pd.read_json('/content/drive/My Drive/kakao_arena/val_song0.json', encoding='utf-8')

In [18]:
val_song0.head()

,tags,id,plylst_title,songs,like_cnt,updt_date,new_tag,original_tag
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000,[앨리스테이블],[]
8,[스트레스],80810,리듬타면서 빡시게 운동하자스트레스 날리자,[],127,2017-02-09 17:33:45.000,"[리듬타면서, 빡시게, 운동하자스트레스]",[날리자]
9,[],142007,기분 좋은 재즈와 함께 만드는 달달한 하루,[],0,2015-06-22 09:11:02.000,"[재즈와, 만드는]","[기분, 좋은, 함께, 달달한, 하루]"
17,[생각나],2380,다시 생각나는 그 사람,[],6,2016-03-19 23:53:20.000,[],"[다시, 생각나는, 그, 사람]"
20,"[피아노, 이루마, 메로디]",149069,불면증엔 아름다운 뉴에이지 곡들,[],11,2019-12-15 19:15:21.000,[불면증엔],"[아름다운, 뉴에이지, 곡들]"


In [19]:
tokenizer = KhaiiiApi()
token_tag = [[get_token(x, tokenizer) for x in i ] for i in val_song0["new_tag"] ]

In [20]:
token_tag2 = []
for i in range(0,len(token_tag)):
  token_tag2.append(flatten(token_tag[i]))

In [21]:
#train["low_tags"]=lower(train["tags"])
#train_lower_tags_set = set(flatten(train["low_tags"]))

In [22]:
len(train_lower_tags_set)

29160

In [23]:
val_song0['origin_khai_tag'] = token_tag2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
val_song0.head()

,tags,id,plylst_title,songs,like_cnt,updt_date,new_tag,original_tag,origin_khai_tag
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000,[앨리스테이블],[],"[(앨리스테, NNP), (이블, NNG)]"
8,[스트레스],80810,리듬타면서 빡시게 운동하자스트레스 날리자,[],127,2017-02-09 17:33:45.000,"[리듬타면서, 빡시게, 운동하자스트레스]",[날리자],"[(리, NNG), (듬타, VV), (면서, EC), (빡, VV), (시, NN..."
9,[],142007,기분 좋은 재즈와 함께 만드는 달달한 하루,[],0,2015-06-22 09:11:02.000,"[재즈와, 만드는]","[기분, 좋은, 함께, 달달한, 하루]","[(재즈, NNG), (와, JC), (만들, VV), (는, ETM)]"
17,[생각나],2380,다시 생각나는 그 사람,[],6,2016-03-19 23:53:20.000,[],"[다시, 생각나는, 그, 사람]",[]
20,"[피아노, 이루마, 메로디]",149069,불면증엔 아름다운 뉴에이지 곡들,[],11,2019-12-15 19:15:21.000,[불면증엔],"[아름다운, 뉴에이지, 곡들]","[(불면증, NNG), (에, JKB), (ㄴ, JX)]"


In [25]:
val_song0['origin_khai_tag_not'] = val_song0['origin_khai_tag'].map(lambda x: list(filter(lambda x: x[0] not in train_lower_tags_set, x)))
val_song0['origin_khai_tag'] = val_song0['origin_khai_tag'].map(lambda x: list(filter(lambda x: x[0] in train_lower_tags_set, x)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [26]:
val_song0[val_song0['origin_khai_tag'].map(len) != 0]

,tags,id,plylst_title,songs,like_cnt,updt_date,new_tag,original_tag,origin_khai_tag,origin_khai_tag_not
8,[스트레스],80810,리듬타면서 빡시게 운동하자스트레스 날리자,[],127,2017-02-09 17:33:45.000,"[리듬타면서, 빡시게, 운동하자스트레스]",[날리자],"[(시, NNG), (운동, NNG), (하, XSV)]","[(리, NNG), (듬타, VV), (면서, EC), (빡, VV), (게, EC..."
9,[],142007,기분 좋은 재즈와 함께 만드는 달달한 하루,[],0,2015-06-22 09:11:02.000,"[재즈와, 만드는]","[기분, 좋은, 함께, 달달한, 하루]","[(재즈, NNG), (와, JC)]","[(만들, VV), (는, ETM)]"
20,"[피아노, 이루마, 메로디]",149069,불면증엔 아름다운 뉴에이지 곡들,[],11,2019-12-15 19:15:21.000,[불면증엔],"[아름다운, 뉴에이지, 곡들]","[(불면증, NNG), (에, JKB)]","[(ㄴ, JX)]"
35,[],65114,사랑그리고이별,[],6,2010-10-27 10:34:34.000,[사랑그리고이별],[],"[(별, NNG)]","[(사랑그, NNG), (리고이, MAG)]"
42,[새벽],8253,하루 끝 잔잔한 노래로 기분 좋게 잠에 들기,[],8,2019-06-08 23:48:41.000,"[좋게, 잠에, 들기]","[하루, 끝, 잔잔한, 노래로, 기분]","[(좋, VA), (잠, NNG), (에, JKB)]","[(게, EC), (들, VV), (기, ETN)]"
...,...,...,...,...,...,...,...,...,...,...
22967,[분위기],36145,한국 갬성 있는 노래들만 모았다,[],12,2020-02-12 13:50:03.000,"[있는, 노래들만, 모았다]","[한국, 갬성]","[(노래, NNG), (만, JX), (다, EC)]","[(있, VV), (는, ETM), (들, XSN), (모으, VV), (았, EP)]"
22969,"[감성, 신나는]",74670,노래 하나로 설레는 그런 발랄함,[],2,2020-03-11 00:55:41.000,"[그런, 발랄함]","[노래, 하나로, 설레는]","[(발랄, XR), (하, XSA)]","[(그런, MM), (ㅁ, ETN)]"
22991,[],32537,컨트리 황제 조니 캐시가 선 레코드 시절 발표한 초기 대표작,[],28,2019-06-17 14:22:48.000,"[조니, 캐시가, 발표한, 대표작]","[컨트리, 황제, 선, 레코드, 시절, 초기]","[(하, XSV)]","[(조니, NNP), (캐시, NNP), (가, JKS), (발표, NNG), (ㄴ..."
22992,[사랑],32812,옷차림이 가벼워질때 부담없이 듣는음악,[],8,2015-05-08 00:09:01.000,"[옷차림이, 가벼워질때]","[부담없이, 듣는음악]","[(이, JKS), (ㄹ, ETM)]","[(옷차림, NNG), (가볍, VA), (어, EC), (지, VX), (때, N..."


In [27]:
using_pos = ['NNG','SL','NNP','MAG','SN','XR']  # 일반 명사, 외국어, 고유 명사, 일반 부사, 숫자, 형용사??(잔잔)
val_song0['origin_khai_tag'] = val_song0['origin_khai_tag'].map(lambda x: list(filter(lambda x: x[1] in using_pos, x)))
val_song0['origin_khai_tag'] = val_song0['origin_khai_tag'].map(lambda x: [tag[0] for tag in x])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
val_song0[val_song0['origin_khai_tag'].map(len) != 0]

,tags,id,plylst_title,songs,like_cnt,updt_date,new_tag,original_tag,origin_khai_tag,origin_khai_tag_not
8,[스트레스],80810,리듬타면서 빡시게 운동하자스트레스 날리자,[],127,2017-02-09 17:33:45.000,"[리듬타면서, 빡시게, 운동하자스트레스]",[날리자],"[시, 운동]","[(리, NNG), (듬타, VV), (면서, EC), (빡, VV), (게, EC..."
9,[],142007,기분 좋은 재즈와 함께 만드는 달달한 하루,[],0,2015-06-22 09:11:02.000,"[재즈와, 만드는]","[기분, 좋은, 함께, 달달한, 하루]",[재즈],"[(만들, VV), (는, ETM)]"
20,"[피아노, 이루마, 메로디]",149069,불면증엔 아름다운 뉴에이지 곡들,[],11,2019-12-15 19:15:21.000,[불면증엔],"[아름다운, 뉴에이지, 곡들]",[불면증],"[(ㄴ, JX)]"
35,[],65114,사랑그리고이별,[],6,2010-10-27 10:34:34.000,[사랑그리고이별],[],[별],"[(사랑그, NNG), (리고이, MAG)]"
42,[새벽],8253,하루 끝 잔잔한 노래로 기분 좋게 잠에 들기,[],8,2019-06-08 23:48:41.000,"[좋게, 잠에, 들기]","[하루, 끝, 잔잔한, 노래로, 기분]",[잠],"[(게, EC), (들, VV), (기, ETN)]"
...,...,...,...,...,...,...,...,...,...,...
22963,"[이별, 회상]",144416,공부는 하기싫고 너는 생각 나서 듣는 노래,[],442,2015-06-08 12:32:34.000,"[공부는, 하기싫고, 너는, 나서]","[생각, 듣는, 노래]",[공부],"[(는, JX), (기, ETN), (싫, VA), (고, EC), (는, JX),..."
22964,[회상],117752,추억이 된 이름을 불러보는 시간,[],2,2015-01-28 10:12:30.000,"[된, 이름을, 불러보는]","[추억이, 시간]",[이름],"[(되, VV), (ㄴ, ETM), (을, JKO), (부르, VV), (어, EC..."
22967,[분위기],36145,한국 갬성 있는 노래들만 모았다,[],12,2020-02-12 13:50:03.000,"[있는, 노래들만, 모았다]","[한국, 갬성]",[노래],"[(있, VV), (는, ETM), (들, XSN), (모으, VV), (았, EP)]"
22969,"[감성, 신나는]",74670,노래 하나로 설레는 그런 발랄함,[],2,2020-03-11 00:55:41.000,"[그런, 발랄함]","[노래, 하나로, 설레는]",[발랄],"[(그런, MM), (ㅁ, ETN)]"


In [29]:
val_song0[val_song0['id'] == 14477]

,tags,id,plylst_title,songs,like_cnt,updt_date,new_tag,original_tag,origin_khai_tag,origin_khai_tag_not
18510,[],14477,New크리스마스,[],6,2013-11-10 19:06:47.000,[New크리스마스],[],[],"[(Ne, SL), (w크리, NNP), (스마스, NNG)]"


In [30]:
val_song0['lower_tags'] =  val_song0['original_tag'] + val_song0['origin_khai_tag'] + lower(val_song0['tags'])
val_song0['lower_tags'] = val_song0['lower_tags'].map(set).map(list)
val_song0 =  val_song0.drop(['origin_khai_tag_not', 'origin_khai_tag', 'original_tag','new_tag'], axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [31]:
val_song0[val_song0['tags'].map(len) == 0]

,tags,id,plylst_title,songs,like_cnt,updt_date,lower_tags
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000,[]
9,[],142007,기분 좋은 재즈와 함께 만드는 달달한 하루,[],0,2015-06-22 09:11:02.000,"[하루, 좋은, 재즈, 기분, 달달한, 함께]"
35,[],65114,사랑그리고이별,[],6,2010-10-27 10:34:34.000,[별]
57,[],87700,마쉬멜로우같은 멜로우한 음악,[],6,2016-01-14 10:19:30.000,"[음악, 멜로, 멜로우한, 우]"
71,[],35271,공부와 독서를 위한 Newage,[],10,2020-01-17 15:46:20.000,"[공부, 위한, 독서를]"
...,...,...,...,...,...,...,...
22903,[],140513,10년이 지나 들어도 좋은 감성 Ballad,[],405,2016-01-11 10:58:05.000,"[감성, 들어도, 10, 좋은]"
22920,[],124704,가사의 의미와 뜻은모른다 오직 멜로디로만 선곡한 팝송,[],27,2016-02-05 12:31:59.000,"[오직, 가사, 선곡, 멜로디, 팝송, 의미]"
22981,[],13045,카페 느낌 샹송,[],38,2011-07-12 00:58:39.000,"[느낌, 카페, 샹송]"
22991,[],32537,컨트리 황제 조니 캐시가 선 레코드 시절 발표한 초기 대표작,[],28,2019-06-17 14:22:48.000,"[황제, 선, 시절, 컨트리, 초기, 레코드]"


In [32]:
val_song0.head(40)

,tags,id,plylst_title,songs,like_cnt,updt_date,lower_tags
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000,[]
8,[스트레스],80810,리듬타면서 빡시게 운동하자스트레스 날리자,[],127,2017-02-09 17:33:45.000,"[스트레스, 날리자, 운동, 시]"
9,[],142007,기분 좋은 재즈와 함께 만드는 달달한 하루,[],0,2015-06-22 09:11:02.000,"[하루, 좋은, 재즈, 기분, 달달한, 함께]"
17,[생각나],2380,다시 생각나는 그 사람,[],6,2016-03-19 23:53:20.000,"[생각나는, 생각나, 사람, 그, 다시]"
20,"[피아노, 이루마, 메로디]",149069,불면증엔 아름다운 뉴에이지 곡들,[],11,2019-12-15 19:15:21.000,"[곡들, 불면증, 이루마, 피아노, 메로디, 뉴에이지, 아름다운]"
35,[],65114,사랑그리고이별,[],6,2010-10-27 10:34:34.000,[별]
40,"[발렌타인데이, 달달한, 사랑노래, 고백]",80682,발렌타인 데이 달콤한 초콜릿 매장 음악,[],27,2019-05-13 15:46:52.000,"[사랑노래, 달콤한, 발렌타인데이, 매장, 초콜릿, 달달한, 고백, 음악, 발렌타인]"
42,[새벽],8253,하루 끝 잔잔한 노래로 기분 좋게 잠에 들기,[],8,2019-06-08 23:48:41.000,"[하루, 노래로, 기분, 잠, 끝, 새벽, 잔잔한]"
43,"[테일러스위프트, 팝의여왕, 컨트리팝]",118572,테일러 스위프트 전곡 모음,[],25,2019-11-24 12:19:01.000,"[컨트리팝, 테일러스위프트, 전곡, 모음, 팝의여왕]"
45,[감성],81994,한국RB I 그루비한 트렌디한 느낌있는,[],14,2020-03-24 11:26:33.000,"[트렌디한, 감성, 느낌있는, 그루비한, 한국]"


In [35]:
val_song0.to_json("/content/drive/My Drive/kakao_arena/val_khaiii.json")